In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
pth = os.path.join('..', '..', '..')
pth_in = os.path.join(pth, 'data')
pth_data = os.path.join(pth_in, 'placodes')
pth_grn = os.path.join(pth_in, 'scenic', 'E8')
pth_out = os.path.join(pth, 'figures', 'E8')

In [ ]:
adata = sc.read_h5ad(os.path.join(pth_data, 'E8.h5ad'))
cat = adata.var.cluster_annotations.to_dict(); adata

In [ ]:
# load regulons
fn = os.path.join(pth_grn, 'regulons.gmt')
reg = pd.read_csv(fn, header = None)

# construct adjacency matrix
A = pd.DataFrame(columns = adata.var_names)
src = reg[0].str[:-3]
tgt = reg[4].str[1:].str.split('\t')
for i in src.index:
    A.loc[src.loc[i]] = 0
    A.loc[src.loc[i], tgt.loc[i]] = 1

In [ ]:
name = 'Adenohypophyseal'
g = A.index[A.index.map(cat) == name]
A_plt = A.loc[g, g].copy()

# graph — connected components
np.fill_diagonal(A_plt.values, 0)
msk = ((A_plt.sum() + A_plt.T.sum()) > 0)
A_plt = A_plt.loc[msk, msk]
G = nx.from_pandas_adjacency(
    A_plt, create_using = nx.DiGraph)
pos = nx.kamada_kawai_layout(G)

# plot
nc, fc, ns, fs = 'b', 'w', 100, 7
s = ns * A_plt.index.str.len()
fig, ax = plt.subplots(1, 1, figsize = (3, 3))
nx.draw_networkx_nodes(
    G, pos,
    node_size = s,
    node_color = nc,
    ax = ax)
nx.draw_networkx_edges(
    G, pos,
    node_size = s,
    width = .8,
    ax = ax)
texts = nx.draw_networkx_labels(
    G, pos,
    font_size = fs,
    font_color = fc,
    ax = ax)
for t in texts.values():
    t.set_fontstyle('italic')
ax.set_axis_off()
ax.set_title(name, size = 11)
fn = os.path.join(pth_out, f'{name}.pdf')
plt.savefig(fn, bbox_inches = 'tight')